In [13]:
import pandas as pd

In [14]:
#df = pd.read_csv("kids1.csv")
# df=pd.read_csv("kidsInMindSubtitles2004.csv")
# df=pd.read_csv("/content/drive/MyDrive/Classroom/kidsInMindSubtitles2004 (2).csv")
df=pd.read_csv("kidsInMindSubtitles2004 (2).csv")
df.shape

(2004, 15)

In [15]:
ds=df

In [16]:
int(len(df)*0.9)

1803

In [17]:
df['subtitles'] = df['subtitles'].apply(lambda x: x.replace('/',''))
df['subtitles'] = df['subtitles'].apply(lambda x: x.replace('\xa0',''))

In [18]:
df1 = df.iloc[:1803,:]
df2 = df.iloc[1803:,:]

In [19]:
pip install transformers


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
class CustomDataset(Dataset):
    def __init__(self, texts, summaries, targets, text_gen_kwargs):
        self.texts = texts
        self.summaries = summaries
        self.targets = targets
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
        self.text_gen_kwargs = text_gen_kwargs

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        input_text = self.texts[idx]
        summary_text = self.summaries[idx]
        target = torch.tensor(self.targets[idx]).float()

        encoded_inputs = self.tokenizer(
            input_text,
            max_length=self.text_gen_kwargs['max_length'],
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        encoded_summaries = self.tokenizer(
            summary_text,
            max_length=self.text_gen_kwargs['max_length'],
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoded_inputs['input_ids'].squeeze(0),
            'attention_mask': encoded_inputs['attention_mask'].squeeze(0),
            'summary_ids': encoded_summaries['input_ids'].squeeze(0),
            'target': target
        }


In [23]:
text_gen_kwargs={'max_length': 248,
 'min_length': 56,
 'do_sample': False,
 'early_stopping': True,
 'num_beams': 4,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'repetition_penalty': 1.0,
 'bad_words_ids': None,
 'bos_token_id': 0,
 'pad_token_id': 1,
 'eos_token_id': 2,
 'length_penalty': 2.0,
 'no_repeat_ngram_size': 3,
 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1,
 'decoder_start_token_id': 2,
 'use_cache': True,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'output_attentions': False,
 'output_hidden_states': False,
 'output_scores': False,
 'return_dict_in_generate': False,
 'forced_bos_token_id': 0,
 'forced_eos_token_id': 2,
 'remove_invalid_values': False}

In [24]:
class BartForMultiTask(torch.nn.Module):
    def __init__(self, text_gen_kwargs):
        super(BartForMultiTask, self).__init__()
        self.bart_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
        self.dropout = torch.nn.Dropout(0.1)
        self.regression_head = torch.nn.Linear(self.bart_model.config.hidden_size, 1)
        self.linear1 = nn.Linear(self.bart_model.config.d_model, 512)
        self.linear2 = nn.Linear(512, 1)
        self.text_gen_kwargs = text_gen_kwargs

    def forward(self, input_ids, attention_mask, summary_ids):
#         print("HI there!")
#         print(input_ids)
        outputs = self.bart_model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=summary_ids, output_hidden_states=True)

        hidden_state = outputs.decoder_hidden_states[-1]
        linear1_output = self.linear1(hidden_state[:, 0])
        regression_output = self.linear2(self.dropout(linear1_output))
        
#         print(outputs.logits.shape)
        return outputs.logits, regression_output.squeeze(-1)

    def generate_text(self, input_ids, attention_mask):
        with torch.no_grad():
            generated_ids = self.bart_model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                **self.text_gen_kwargs
            )
        return generated_ids




In [25]:
from transformers import BartTokenizer

# Create the tokenizer object
# tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
tokenizer =AutoTokenizer.from_pretrained('facebook/bart-large-cnn')


In [26]:
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

For the Entire Batch calculating Rouge

In [27]:
texts = df1['subtitles'].tolist()
summaries = df1['Language_Description'].tolist()
labels_regression = df1['Language'].tolist()  # Regression labels

In [28]:
train_dataset = CustomDataset(texts, summaries, labels_regression,text_gen_kwargs)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last = True)

In [29]:
val_texts = df2['subtitles'].tolist()
val_summaries = df2['Language_Description'].tolist()
val_targets = df2['Language'].tolist()  # Regression labels

In [30]:
val_dataset = CustomDataset(val_texts, val_summaries, val_targets,text_gen_kwargs)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=True, drop_last = True)

In [31]:
from fastai.text.all import *

In [32]:
model = BartForMultiTask(text_gen_kwargs)

In [33]:
dls = DataLoaders.from_dsets(train_dataset, val_dataset, bs=2, drop_last=True)

In [34]:
xb, yb, *_= dls.one_batch()
print('Input batch:', xb)
print('Target batch:', yb)

Input batch: input_ids
Target batch: attention_mask


In [35]:
print(xb)

input_ids


In [36]:
batch = dls.one_batch()
input_ids = batch['input_ids']
attention_mask = batch['attention_mask']
target = batch['target']

print('Input batch - input_ids:', input_ids)
print('Input batch - attention_mask:', attention_mask)
print('Target batch:', target)


Input batch - input_ids: tensor([[    0, 41552,   118, 15698,  1000,   328, 17514,     6,  1577,   328,
         41552,   118, 15698, 13368,     6,   313,     6,   912,   310,   179,
           108,   198,     4,  2709,   588,     6,   313,     4, 50118,  1185,
          2972, 16779,   108,    50,    99,   116,    12, 28696,   118, 15698,
         14783,     6,   905,    18,   213,     4, 28696,   118, 15698, 50118,
            12,  8976,     4, 14783,     6, 11380,     6, 11380,     4, 50118,
         22817,    15,     4, 10050,  9120,   328, 41552,   118, 15698, 28062,
         19774,     6,    70,    47,  2204,  4825,  9945,   734, 28696,   118,
         15698,   463,    70,    47,  2418,  3279,   268,     4,   713,    16,
           211,     4,   863,     4, 36563, 18539,  1686,   734, 31647,    47,
             6,   120,    15,     5,  1929,   734,   463, 12800,  6612, 50118,
          6968,    64,    75, 12800,  5988,   955, 43326,    71,   452,     6,
             5,  6081, 5011

In [37]:
#print(batch)

In [38]:
import numpy as np
from transformers import BartTokenizer

In [39]:

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

In [40]:
outputs=model(batch['input_ids'],batch['attention_mask'],batch['summary_ids'])

In [41]:
output_tensor = outputs[0]

In [42]:
output_array = output_tensor.detach().cpu().numpy()

In [43]:
predicted_ids = np.argmax(output_array,axis=2)

In [44]:
loss = model(batch['input_ids'],batch['attention_mask'],batch['summary_ids'])

In [45]:
loss[0].shape

torch.Size([2, 248, 50264])

In [46]:
loss[1].shape

torch.Size([2])

In [47]:
loss[0].view(-1, loss[0].size(-1)).shape

torch.Size([496, 50264])

In [48]:
for d in dls:
    break

In [49]:
from fastai.text.all import *
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from rouge_score import rouge_scorer
from bert_score import score as bert_score

In [50]:
summarization_loss_func = nn.CrossEntropyLoss() 
regression_loss_func = nn.MSELoss()

In [51]:
summarization_loss_weight = 2.99
regression_loss_weight = 1


In [53]:
def multi_task_loss(preds, targets,summary_ids):

    summarization_loss = summarization_loss_weight * summarization_loss_func(preds[0].view(-1, preds[0].size(-1)), summary_ids.view(-1))
    regression_loss = regression_loss_weight * regression_loss_func(preds[1], targets)

    
    total_loss = summarization_loss + regression_loss
    
    total_loss_ = total_loss.detach().cpu().item()
    with open("experiment_9.txt", "a") as file:
        file.write(str(total_loss_)+"\n")
    return total_loss


In [54]:
learn = Learner(dls, model, loss_func=multi_task_loss)

In [ ]:
learn.fit_one_cycle(15, lr_max=2e-6)

In [ ]:
learn.save('MultiTask_Bart_model')

In [57]:
loaded_dict = torch.load('models/MultiTask_Bart_model.pth')

In [58]:
model_state_dict = loaded_dict['model']

In [59]:
model = BartForMultiTask(text_gen_kwargs)

# Load the state dict into the model
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [60]:
model.eval()

BartForMultiTask(
  (bart_model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50264, 1024, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50264, 1024, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
        (layers): ModuleList(
          (0): BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_fe

In [105]:
header=["Subtitles","Language_Description","Generated_Summary","Language_Score","Generated_Langauge_Score"]
Generated_Summary=[]
Generated_Language_Score=[]
subtitles=ds.subtitles

for sub in subtitles[1803:]:
    inputs = tokenizer(sub, truncation=True, padding=True, return_tensors="pt")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    summary_ids = model.generate_text(input_ids, attention_mask)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     outputs = model(input_ids, attention_mask,summary_ids)
#     regression_output = outputs[1].item()
    Generated_Summary.append(summary)
#     Generated_Language_Score.append(regression_output)
        
        
        
    

In [106]:
len(Generated_Summary)

201

In [107]:
lang=ds['Language_Description'].tolist()

In [108]:
df4_modelepochs = pd.DataFrame({'Language_Description':lang[1803:],"Generated_Summary":Generated_Summary})

In [111]:
df4_modelepochs.to_csv('SummariesFormodel_9.csv')

In [135]:
df_temp=pd.read_csv('SummariesFormodel_9.csv')

In [136]:
len(df_temp)

201

In [164]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


summaries = df_temp['Language_Description'].tolist()
references = df_temp['Generated_Summary'].tolist()
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for summary, reference in zip(summaries, references):
    scores = scorer.score(summary, reference)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

average_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
average_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
average_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)


In [165]:
print("average_rouge1_score: ",average_rouge1_score)
print("average_rouge2_score: ",average_rouge2_score)
print("average_rougeL_score: ",average_rougeL_score)

average_rouge1_score:  0.01389144820500645
average_rouge2_score:  0.0007391721454826935
average_rougeL_score:  0.011095096804847231


In [166]:
len(summaries)

201

In [167]:
import bert_score

model_type = 'facebook/bart-large-cnn'

P, R, F1 = bert_score.score(summaries, references, lang='en', model_type=model_type)
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

print("BERTScore Precision:", bertscore_precision)
print("BERTScore Recall:", bertscore_recall)
print("BERTScore F1:", bertscore_f1)


BERTScore Precision: 0.35522180795669556
BERTScore Recall: 0.2872545123100281
BERTScore F1: 0.30642372369766235
